# Example of slurm job creation and submission (on levante)

## Structure

0. [Install the following:]()

    - [dask]() (conda)

    - [dask_jobqueue]() (pip)

    - [os, re]()

1. [Importing the job module](#1-importing-the-job-module)

2. [Creating the Slurm Job on LEVANTE](#2-creating-the-slurm-job-on-levante)

    2.1 [Creating the Job with manual selection of cores, memory, and walltime](#21-creating-the-job-with-manual-selection-of-cores-memory-and-walltime)

    2.2 [Creating the Job with exclusive node access](#22-creating-the-job-with-exclusive-node-access)

    2.3 [Creating the Job with maxumum availibale resources per the node](#23-creating-the-job-with-maxumum-availibale-resources-per-the-node)

    2.4 [Redirecting the SLURM output to /any/path/you/want](#24-redirecting-the-slurm-output-to-anypathyouwant)

3. [Creating and Submitting the Job to the SLURM queue on Lumi](#3-creating-and-submitting-the-job-to-the-slurm-queue-on-lumi)
    
4. [Canceling the Slurm Job](#4-canceling-the-slurm-job)

    3.1 [Canceling all jobs of the user](#41-cancelling-all-jobs-of-user)

    3.2 [Canceling the specific Job](#42-canceling-specific-job)

## 1. Importing the job module

##### The `job` module contains the following functions: `squeue`, `job`, `max_resources_per_node`, `scancel`, which allows us to create and operate the Slurm Job

In [2]:
from aqua.slurm import slurm

## 2. Creating the Slurm Job on LEVANTE

### 2.1 Creating the Job with manual selection of cores, memory, and walltime

The function `slurm.job()` allows to create a job in the slurm queue on levante. Some options are available.

In [12]:
slurm.job()

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G
#SBATCH -t 02:30:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out

/work/bb1153/b382289/mambaforge/envs/aqua/bin/python -m distributed.cli.dask_worker tcp://136.172.124.5:33807 --nthreads 1 --memory-limit 9.31GiB --name dummy-name --nanny --death-timeout 60



/work/bb1153/b382289/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 32937 instead
  warnings.warn(


We can check the status of created Job in the queue using the function `squeue()`

In [13]:
slurm.squeue()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           5233510   compute dask-wor  b382289 PD       0:00      1 (Priority)


0

By default, the Job has the following attributes:

- `exclusive=False`: If True, the job will be submitted asking for exclusive access to the node.
- `max_resources=False`: If True, the job will be submitted asking for the maximum resources available on the node.
- `cores=1`: number of cores per socket.
- `memory="10 GB"`: real memory required per node.
- `queue="compute"`: queue/partition to which SLURM submit the job.
- `walltime="02:30:00"`: duration of the allocation.
- `jobs=1`: factor of assignment scaling across multiple nodes.
- `account=bb1153`: account that submits the job. It is the project id on levante.
- `path_to_output="."`: path where log, err and output files are stored.

 
If you want to use a different amount of cores, memory, wall time, jobs, or a different queue, you can specify it as an argument of function:

In [15]:
slurm.job(cores=8, memory="50 GB", queue="interactive", walltime='00:30:00', jobs=1)

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p interactive
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=8
#SBATCH --mem=47G
#SBATCH -t 00:30:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out

/work/bb1153/b382289/mambaforge/envs/aqua/bin/python -m distributed.cli.dask_worker tcp://136.172.124.5:40169 --nthreads 2 --nworkers 4 --memory-limit 11.64GiB --name dummy-name --nanny --death-timeout 60



/work/bb1153/b382289/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40665 instead
  warnings.warn(


### 2.2. Creating the Job with exclusive node access

##### The function has an argument `exclusive`, which is False by default.  If we set the argument to True, we will get exclusive access to the node.


In [18]:
slurm.job(exclusive=True)

/work/bb1153/b382289/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45309 instead
  warnings.warn(


#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=1
#SBATCH --mem=10G
#SBATCH -t 02:30:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out
#SBATCH --get-user-env
#SBATCH --exclusive

/work/bb1153/b382289/mambaforge/envs/aqua/bin/python -m distributed.cli.dask_worker tcp://136.172.124.5:34597 --nthreads 1 --memory-limit 9.31GiB --name dummy-name --nanny --death-timeout 60



## Important! 

The `exclusive` argument DOES NOT automatically provide us the maximum available memory, number of cores, and walltime!
It only provide you the exclusive usage of the node, meaning that no other job can run at the same time on the same node.

In [21]:
slurm.job(exclusive=True, cores=256, memory="500 GB", queue = "interactive", walltime='12:00:00', jobs=1)

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p interactive
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=256
#SBATCH --mem=466G
#SBATCH -t 12:00:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out
#SBATCH --get-user-env
#SBATCH --exclusive

/work/bb1153/b382289/mambaforge/envs/aqua/bin/python -m distributed.cli.dask_worker tcp://136.172.124.5:42651 --nthreads 16 --nworkers 16 --memory-limit 29.10GiB --name dummy-name --nanny --death-timeout 60



/work/bb1153/b382289/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36309 instead
  warnings.warn(


### 2.3. Creating the Job with maxumum availibale resources per the node

The function has an argument `max_resources_per_node`, which is `False` by default. If we set the argument to `True`, the number of cores, memory, and walltime will equal the maximum allowed by the selected queue/partition.

In [24]:
slurm.job(max_resources=True)

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=256
#SBATCH --mem=235G
#SBATCH -t 8:00:00
#SBATCH --error=./slurm/logs/dask-worker-%j.err
#SBATCH --output=./slurm/output/dask-worker-%j.out

/work/bb1153/b382289/mambaforge/envs/aqua/bin/python -m distributed.cli.dask_worker tcp://136.172.124.5:38707 --nthreads 16 --nworkers 16 --memory-limit 14.63GiB --name dummy-name --nanny --death-timeout 60



/work/bb1153/b382289/mambaforge/envs/aqua/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46233 instead
  warnings.warn(


With the argument `max_resources_per_node=True`, the function `max_resources_per_node()` automatically extracts the following information about the resources in any queue:

 - Size of memory per node in Gigabytes: `max_memory`, 

 - Maximum time for any job in the format "days-hours:minutes:seconds: `max_walltime`

 - Number of CPUs per node: `max_cpus`, 

 - Number of sockets per node: `max_sockets`

 - Number of cores per socket: `max_cores`, 
 
 - Number of threads per core: `max_threads`

The function can be also used alone to have info about a specific queue/partition.
For example:

In [27]:
slurm.max_resources_per_node('compute')

('251.3671875 GB', '8:00:00', '256', '8', '16', '2')

In [28]:
slurm.max_resources_per_node('interactive')

('502.9296875 GB', '12:00:00', '256', '8', '16', '2')

### 2.4 Redirecting the SLURM output to `/any/path/you/want`

Slurm Job writes by default
    - the errors into `./slurm/logs` directory 
    - the output into `./slurm/output/` directory

If folders `/slurm`,  `/slurm/output`,  `/slurm/logs` do not exist, the function will create them automatically. 

The user can specify a custom path to redirect the SLURM outputs with the `path_to_output` option:

In [ ]:
slurm.job(path_to_output='/any/path/you/want')

## 3. Creating and Submitting the Job to the SLURM queue on Lumi (Under development)

You may need to change the account name if you want to create the Slurm Job on Lumi with the function `job()`. Currently on levante, it  is `account="bb1153"` and it is valid for the whole project.

In [ ]:
slurm.job(account='Your_Lumi_account_name')

## 4. Canceling the Slurm Job

### 4.1 Cancelling all jobs of user

In [18]:
slurm.scancel()

### 4.2 Canceling specific job

##### Knowing the Job_ID, you can cancel your Job in the queue. For exaple, you can find your Job_ID using the function `squeue().` 

In [5]:
Job_ID = 4929434
slurm.scancel(Job_ID)

##### Checking the status of canceled Job

In [19]:
slurm.squeue()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


0